In [1]:
# import initializer
import kernel_init

# import packages
from texturedar.models import *
from azure.storage.blob import BlobServiceClient
import datetime as dt
from django.contrib.auth.models import User
import json
import os
import pandas as pd
import pytz as tz
import sys
from tqdm import tqdm
import pathlib
import numpy as np


In [2]:
# Setup paths to production data
PRE_SPEC_PATH = pathlib.Path(
    r"C:\Users\hsmith\OneDrive - THORLABS Inc\Documents - Thorlabs Spectral Works\Production\TAR\Production Data\AR Data\Pre"
)

POST_SPEC_PATH = pathlib.Path(
    r"C:\Users\hsmith\OneDrive - THORLABS Inc\Documents - Thorlabs Spectral Works\Production\TAR\Production Data\AR Data\Post"
)

PRE_IMG_PATH = pathlib.Path(
    r"C:\Users\hsmith\OneDrive - THORLABS Inc\Documents - Thorlabs Spectral Works\Production\TAR\Production Data\Images\Pre"
)

POST_IMG_PATH = pathlib.Path(
    r"C:\Users\hsmith\OneDrive - THORLABS Inc\Documents - Thorlabs Spectral Works\Production\TAR\Production Data\Images\Post"
)

In [12]:
# upload pre spec files and update database records
## pre_ccs = []
## pre_flame = []
## pre_spec_files = [
##     p for p in os.listdir(PRE_SPEC_PATH)
##     if 'binned' not in p.lower() and 'flame_t' not in p.lower()
## ]
## for f in pre_spec_files:
##     if 'CCS' in f.upper():
##         pre_ccs.append(os.path.join(PRE_SPEC_PATH, f))
##     elif 'FLAME' in f.upper():
##         pre_flame.append(os.path.join(PRE_SPEC_PATH, f))
## 
## 
## for serial, ccsfile, flamefile in zip([50, 53], pre_ccs, pre_flame):
##     with open(ccsfile, 'r') as f:
##         row = f.readline()
##         wv_300 = []
##         wv_500 = []
##         THRESH = 10
##         while row:
##             row = row.strip().replace('\n','').split('\t')
##             try:
##                 wv = float(row[0])
##                 dark = float(row[1])
##                 ref = float(row[2])
##                 sample = float(row[3])
## 
##                 if (300.0-THRESH) < wv < (300.0+THRESH):
##                     wv_300.append((sample - dark)/(ref - dark))
##                 elif (500.0-THRESH) < wv < (500.0+THRESH):
##                     wv_500.append((sample - dark)/(ref - dark))
## 
##             except:
##                 pass
##             row = f.readline()
## 
##         mean_300 = np.mean(wv_300)
##         mean_500 = np.mean(wv_500)
##         print(
##             f"Calculating average at 300nm and 500nm --->\n\n\t{round(mean_300, 4)} and {round(mean_500, 4)}\n\n")
## 
## 
##     with open(flamefile, 'r') as f:
##         row = f.readline()
##         wv_1385 = []
##         while row:
##             row = row.strip().replace('\n','').split('\t')
##             try:
##                 wv = float(row[0])
##                 dark = float(row[1])
##                 ref = float(row[2])
##                 sample = float(row[3])
## 
##                 if (1385.0-THRESH) < wv < (1385.0+THRESH):
##                     wv_1385.append((sample - dark)/(ref - dark))
## 
##             except:
##                 pass
## 
##             row = f.readline()
## 
##         mean_1385 = np.mean(wv_1385)
##         print(f"Calculating average at 1385nm --->\n\n\t{round(mean_1385, 4)}\n\n")
## 
##         part = AR_Part.objects.get(serial=serial)
##         part.trans_300_pre = mean_300
##         part.trans_500_pre = mean_500
##         part.trans_1385_pre = mean_1385
##         part.uv_vis_data_pre = f'ar_data/production/uv_vis_data/pre/{os.path.basename(ccsfile)}'
##         part.ir_data_pre = f'ar_data/production/ir_data/pre/{os.path.basename(flamefile)}'
##         part.save()
## 
## 

In [4]:
# upload pre image files
## pre_img_files = os.listdir(PRE_IMG_PATH)
## serials = [int(s.split(' ')[0]) for s in pre_img_files]
## for s, f in zip(serials, pre_img_files):
##     part = AR_Part.objects.get(serial=s)
##     part.image_data_pre = f'ar_data/production/image_data/pre/{f}'
##     part.save()

In [7]:
# get sample serial no's from file names
samples = []
for f in os.listdir(POST_SPEC_PATH):
    fname = f.split(' ')
    s = int(fname[0])
    if not s in samples:
        samples.append(s)
print(samples)

[1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 50, 51, 52, 53, 54, 55, 56, 8, 43, 44, 45, 46, 47, 48, 49, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84]


In [14]:
# upload post spec files and update database records
## post_ccs = []
## post_flame = []
## post_spec_files = [
##     p for p in os.listdir(POST_SPEC_PATH)
##     if 'binned' not in p.lower() and 'flame_t' not in p.lower()
## ]
## for f in post_spec_files:
##     if 'CCS' in f.upper():
##         post_ccs.append(os.path.join(POST_SPEC_PATH, f))
##     elif 'FLAME' in f.upper():
##         post_flame.append(os.path.join(POST_SPEC_PATH, f))
## 
## print('serial \t | \t ccs  \t | \t  flame')
## print('-'*60)
## for serial, ccsfile, flamefile in zip(samples, post_ccs, post_flame):
##     print(f"{serial}\t{os.path.basename(ccsfile)}\t{os.path.basename(flamefile)}")
##     with open(ccsfile, 'r') as f:
##         row = f.readline()
##         wv_300 = []
##         wv_500 = []
##         wv_700 = []
##         wv_900 = []
##         THRESH = 10
##         while row:
##             row = row.strip().replace('\n','').split('\t')
##             try:
##                 wv = float(row[0])
##                 dark = float(row[1])
##                 ref = float(row[2])
##                 sample = float(row[3])
## 
##                 if (300.0-THRESH) < wv < (300.0+THRESH):
##                     wv_300.append((sample - dark)/(ref - dark))
##                 elif (500.0-THRESH) < wv < (500.0+THRESH):
##                     wv_500.append((sample - dark)/(ref - dark))
##                 elif (700.0-THRESH) < wv < (700.0+THRESH):
##                     wv_700.append((sample - dark)/(ref - dark))
##                 elif (900.0-THRESH) < wv < (900.0+THRESH):
##                     wv_900.append((sample - dark)/(ref - dark))                    
## 
##             except:
##                 pass
##             row = f.readline()
## 
##         mean_300 = np.mean(wv_300)
##         mean_500 = np.mean(wv_500)
##         mean_700 = np.mean(wv_700)
##         mean_900 = np.mean(wv_900)
## 
## 
##     with open(flamefile, 'r') as f:
##         row = f.readline()
##         wv_1100 = []
##         wv_1300 = []
##         wv_1500 = []
##         wv_1650 = []
## 
##         while row:
##             row = row.strip().replace('\n','').split('\t')
##             try:
##                 wv = float(row[0])
##                 dark = float(row[1])
##                 ref = float(row[2])
##                 sample = float(row[3])
## 
##                 if (1100.0-THRESH) < wv < (1100.0+THRESH):
##                     wv_1100.append((sample - dark)/(ref - dark))
##                 elif (1300.0-THRESH) < wv < (1300.0+THRESH):
##                     wv_1300.append((sample - dark)/(ref - dark))
##                 elif (1500.0-THRESH) < wv < (1500.0+THRESH):
##                     wv_1500.append((sample - dark)/(ref - dark))
##                 elif (1650.0-THRESH) < wv < (1650.0+THRESH):
##                     wv_1650.append((sample - dark)/(ref - dark))
## 
##             except:
##                 pass
## 
##             row = f.readline()
## 
##         mean_1100 = np.mean(wv_1100)
##         mean_1300 = np.mean(wv_1300)
##         mean_1500 = np.mean(wv_1500)
##         mean_1650 = np.mean(wv_1650)
## 
##         part = AR_Part.objects.get(serial=serial)
## 
##         part.trans_300_post = mean_300
##         part.trans_500_post = mean_500
##         part.trans_700_post = mean_700
##         part.trans_900_post = mean_900
##         part.trans_1100_post = mean_1100
##         part.trans_1300_post = mean_1300
##         part.trans_1500_post = mean_1500
##         part.trans_1650_post = mean_1650
##         
##         part.uv_vis_data_post = f'ar_data/production/uv_vis_data/post/{os.path.basename(ccsfile)}'
##         part.ir_data_post = f'ar_data/production/ir_data/post/{os.path.basename(flamefile)}'
## 
##         part.save()
## 
## 

In [16]:
# upload post image files
post_img_files = os.listdir(POST_IMG_PATH)
serials = [int(s.split(' ')[0]) for s in post_img_files]
for s, f in zip(serials, post_img_files):
    print(f"{s}\t{f}")
    part = AR_Part.objects.get(serial=s)
    part.image_data_post = f'ar_data/production/image_data/post/{f}'
    part.save()

1	0000001 final.jpg
2	0000002 final.jpg
3	0000003 final.jpg
4	0000004 final.jpg
5	0000005 final.jpg
6	0000006 final.jpg
7	0000007 final.jpg
8	0000008 final.jpg
9	0000009 final.jpg
10	0000010 final.jpg
11	0000011 final.jpg
12	0000012 final.jpg
13	0000013 final.jpg
14	0000014 final.jpg
22	0000022 final.jpg
23	0000023 final.jpg
24	0000024 final.jpg
25	0000025 final.jpg
26	0000026 final.jpg
27	0000027 final.jpg
28	0000028 final.jpg
29	0000029 final.jpg
30	0000030 final.jpg
31	0000031 final.jpg
32	0000032 final.jpg
33	0000033 final.jpg
34	0000034 final.jpg
35	0000035 final.jpg
36	0000036 final.jpg
37	0000037 final.jpg
38	0000038 final.jpg
39	0000039 final.jpg
40	0000040 final.jpg
41	0000041 final.jpg
42	0000042 final.jpg
43	0000043 final.jpg
44	0000044 final.jpg
45	0000045 final.jpg
46	0000046 final.jpg
47	0000047 final.jpg
48	0000048 final.jpg
49	0000049 final.jpg
50	0000050 final.jpg
51	0000051 final.jpg
52	0000052 final.jpg
53	0000053 final.jpg
54	0000054 final.jpg
55	0000055 final.jpg
5